## Download 4 haarcascade models from [openCV github](https://github.com/opencv/opencv/tree/master/data/haarcascades)  

In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import shutil
import random

__Cascade Classifiers 로드__

In [2]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontface.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
glass_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
eyesplit_cascade = cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')

__advanced_eye_detect 함수 정의__

In [3]:
def advanced_eye_detect(img,cascade,info):
    
    ret_val=[]
    
    img_gray =  cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(img_gray)
    
    if len(faces)!=1:
        return ret_val
    
    for (x,y,w,h) in faces:
        roi_gray = img_gray[y:y+h, x:x+w]
        eyes =cascade.detectMultiScale(roi_gray,1.1,4)
    
    if not (len(eyes)==2):    
        if info == 'normal':
            return advanced_eye_detect(img,eyesplit_cascade,'split')
        elif info == 'split':
            return advanced_eye_detect(img,glass_cascade,'glasses')
        elif info == 'glasses':
            return ret_val
        
    ret_val = eyes
    return ret_val

__get_rotated_image 함수 정의__

In [4]:
def get_rotated_image(img,eyes):
    
    eye_1 , eye_2 = eyes

    if eye_1[0] < eye_2[0]:
        left_eye = eye_1
        right_eye = eye_2
    else:
        left_eye = eye_2
        right_eye = eye_1
        
    left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
    right_eye_center = (int(right_eye[0] + (right_eye[2]/2)), int(right_eye[1] + (right_eye[3]/2)))
    
    left_eye_x , left_eye_y = left_eye_center 
    right_eye_x , right_eye_y = right_eye_center

    
    delta_x = right_eye_x - left_eye_x
    delta_y = right_eye_y - left_eye_y
    
    if not delta_x or delta_y:
        return img
    
    angle=np.arctan(delta_y/delta_x)
    angle = (angle * 180) / np.pi
    
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, (angle), 1.0)
    rotated_img = cv2.warpAffine(img, M, (w, h))
  
    return rotated_img

__calculate_rotated_eyes 함수 정의__

In [5]:
def calculate_rotated_eyes(rotated_img,cascade,info):

    ret_val=[]
    
    rotated_gray = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(rotated_gray, 1.1, 4)
    
    if len(faces)!=1:
        return ret_val
    
    for (x,y,w,h) in faces:

        black=np.zeros(rotated_gray.shape,dtype='uint8')
        black[y:y+h, x:x+w]=rotated_gray[y:y+h, x:x+w]

        rotated_eyes =cascade.detectMultiScale(black,1.1,4)    
    
    if len(rotated_eyes)!=2:    
        if info == 'normal':
            return calculate_rotated_eyes(rotated_img,glass_cascade,'glasses')
        elif info == 'glasses':
            return ret_val
    
    ret_val = rotated_eyes
    
    return ret_val

__extract_facial_mask_area 함수 정의__

In [6]:
def extract_facial_mask_area(rotated_img,rotated_eyes):
    
    eye_1 , eye_2 = rotated_eyes

    if eye_1[0] < eye_2[0]:
        left_eye = eye_1
        right_eye = eye_2
    else:
        left_eye = eye_2
        right_eye = eye_1
    
    left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
    right_eye_center = (int(right_eye[0] + (right_eye[2]/2)), int(right_eye[1] + (right_eye[3]/2)))
    
    left_eye_x , left_eye_y = left_eye_center 
    right_eye_x , right_eye_y = right_eye_center
    
    delta_x = right_eye_x - left_eye_x
    delta_y = right_eye_y - left_eye_y
    
    L = np.sqrt(delta_x**2 + delta_y**2)
    xpad_L , xpad_R = int(0.6*L) , int(1.6*L)
    ypad_U , ypad_D = int(0.6*L) , int(1.8*L)
    
    ROI = rotated_img[left_eye_y-ypad_U:left_eye_y+ypad_D,left_eye_x-xpad_L:left_eye_x+xpad_R]
    ROI_resized = cv2.resize(ROI,(120,140))
    mask_area = ROI_resized[50:140,0:120]
    
    return mask_area

__메인 코드__

if full_path ==  
- C:/images/no_mask  
- C:/images/nose_mask  
- C:/images/chin_mask  
- C:/images/full_mask  
***
upper_path = C:/images

In [7]:
upper_path="D:/test"
types_=["no_mask","nose_mask","chin_mask","full_mask"]

In [8]:
lens=[]
for type_ in types_:
    
    img_path = os.path.join(upper_path,type_)
    save_path = os.path.join(upper_path,type_+"_extracted")
    if not os.path.isdir(save_path): os.mkdir(save_path)

    files = os.listdir(img_path)
    image_files = [file for file in files if file.endswith(".png")]
    extracted_cnt=0
    for file in image_files:

        image=cv2.imread(img_path+"/"+file)

        if image.shape[0]<400 or image.shape[1]<400 :
            w,h,c = image.shape
            image = cv2.resize(image,(h*2,w*2))

        eyes = advanced_eye_detect(image,eye_cascade,'normal')

        if len(eyes)!=2:
            continue

        rotated_image = get_rotated_image(image,eyes)
        rotated_eyes = calculate_rotated_eyes(rotated_image,eye_cascade,'normal')

        if len(rotated_eyes)!=2:
            continue

        current_mask_area = extract_facial_mask_area(rotated_image,rotated_eyes)
        extracted_cnt+=1
        cv2.imwrite(os.path.join(save_path,file), current_mask_area) # replace it with your path
    
    lens.append(extracted_cnt)
    print(f"Extracted {extracted_cnt} area with type : {type_}")

Extracted 622 area with type : no_mask
Extracted 371 area with type : nose_mask
Extracted 631 area with type : chin_mask
Extracted 153 area with type : full_mask


__Train | Validate | Test split (7:2:1)__

In [9]:
from sklearn.model_selection import train_test_split
np.random.seed(1120)

split_path = upper_path+'/'+'dataset' # 저장할 경로 (안에 빈 train , test 폴더 먼저 만들어 놓고 실행)
match_size=min(lens)

if not os.path.isdir(split_path):os.mkdir(split_path)
if not os.path.isdir(os.path.join(split_path,"train")):os.mkdir(os.path.join(split_path,"train"))
if not os.path.isdir(os.path.join(split_path,"val")):os.mkdir(os.path.join(split_path,"val"))
if not os.path.isdir(os.path.join(split_path,"test")):os.mkdir(os.path.join(split_path,"test"))

for type_ in types_:
    train_path = os.path.join(os.path.join(split_path,"train"),type_)
    val_path = os.path.join(os.path.join(split_path,"val"),type_)
    test_path = os.path.join(os.path.join(split_path,"test"),type_)
    
    if not os.path.isdir(train_path):os.mkdir(train_path)
    if not os.path.isdir(val_path):os.mkdir(val_path)
    if not os.path.isdir(test_path):os.mkdir(test_path)
    
    files = os.listdir(upper_path+"/"+type_+"_extracted")
    
    image_files = [file for file in files if file.endswith(".png")]
    
    samples = random.sample(image_files,match_size)
    
    train,test = train_test_split(samples,test_size=0.1,random_state=1120)
    train,validate = train_test_split(train,test_size=(2/9),random_state=1120)
    
    for img in train:
        shutil.copy(upper_path+"/"+type_+"_extracted/"+img,train_path+'/'+img)
        
    for img in validate:
        shutil.copy(upper_path+"/"+type_+"_extracted/"+img,val_path+'/'+img)
        
    for img in test:
        shutil.copy(upper_path+"/"+type_+"_extracted/"+img,test_path+'/'+img)
    
print("Split done")

Split done
